In [58]:
import pandas as pd
import numpy as np
from collections import Counter

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


* Loading movie dataset with two columns movie name and it's review

In [ ]:
df_movie = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/moviereviews.csv')
df_movie.head()

,movie,review
0,The Lord of the Rings The Two Towers,remarkable display of fantasy action powerful ...
1,Inception,implanting stealing idea destroy gripping acti...
2,Spiderman Across the spider verse,mind bending wild action sequences intimate em...
3,The Dark Knight,Best live action portrayal beat organized crim...
4,Three colors red,mesmerising friendship turned love profound un...


2. Write a program that identifies top 3 pairs of movies that are very much alike

## Data preprocessing

In [ ]:
#data preprocesing: convering to lower case
df_new = pd.DataFrame()
df_new['movie'] = df_movie['movie'].apply(lambda x: x.lower())
df_new['review'] = df_movie['review'].apply(lambda x: x.lower())
df_new.head()

,movie,review
0,the lord of the rings the two towers,remarkable display of fantasy action powerful ...
1,inception,implanting stealing idea destroy gripping acti...
2,spiderman across the spider verse,mind bending wild action sequences intimate em...
3,the dark knight,best live action portrayal beat organized crim...
4,three colors red,mesmerising friendship turned love profound un...


In [118]:
#remove stop words
stop_words = ['i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've", "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', 'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their', 'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', 'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', 'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', 'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after', 'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further', 'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more', 'most', 'other', 'some', 'such', 'no', 'nor', 'not', 'only', 'own', 'same', 'so', 'than', 'too', 'very', 's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', 've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn', "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn', "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", 'won', "won't", 'wouldn', "wouldn't"]

def keep_word(word):
    return word not in stop_words

lst = []
for i in range(len(df_new)):
  l =  f"{df_new['movie'].iloc[i]} {df_new['review'].iloc[i]}"
  filtered_words = filter(keep_word, l.split())
  lst.append(' '.join(list(filtered_words)))
df_new['movie_processed'] = lst
df_new.drop(['movie','review'], axis = 1, inplace=True)
df_new

,movie_processed
0,lord rings two towers remarkable display fanta...
1,inception implanting stealing idea destroy gri...
2,spiderman across spider verse mind bending wil...
3,dark knight best live action portrayal beat or...
4,three colors red mesmerising friendship turned...
5,happened one night romantic comedy screwball c...
6,mood love neighbors solace bonding affair pred...
7,sunrise blossoming love know chance encounter ...
8,gone wind epic romance greatest romantic film ...
9,eternal sunshine spotless mind length people g...


## Vocabulary

In [119]:
lst = []
for i in range(len(df_new)):
  lst.extend(df_new['movie_processed'].iloc[i].split())
vocabulary = list(set(lst))

## Creating word vector

In [120]:
def word_vector_calculate(movie):
  word_vec = dict.fromkeys(vocabulary,0)
  for word in movie.split(' '):
      word_vec[word]=movie.count(word)
  return word_vec

In [121]:
lst = []
for i in range(len(df_new)):
  l = f"{df_new['movie_processed'].iloc[i]}"
  new = word_vector_calculate(l)
  lst.append(new)
df_word_vec = pd.DataFrame(lst)

In [123]:
df_word_vec.head()

,hobbit,gripping,bending,affair,classic,portrayal,pitiable,go,verse,ottoman,...,bull,mesmerising,romantic,love,eternal,fleeting,jaw,heart,happened,greatest
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
2,0,0,1,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,1,0,1,0,0,0,0,0,0


## Calculating Similarity

In [124]:
df_word_vec.shape[1]

145

In [126]:
def cosine_similarity(x, y):
    return np.dot(x, y) / (np.sqrt(np.dot(x, x)) * np.sqrt(np.dot(y, y)))

In [137]:
dic = {}
for i in range(len(df_word_vec)):
  for j in range(len(df_word_vec)):
    if (i!=j):
      new = cosine_similarity(df_word_vec.iloc[i], df_word_vec.iloc[j])
      if (i,j) and (j,i) not in dic.keys():
        dic[(i,j)] = new
dic

{(0, 1): 0.20801257358446093,
 (0, 2): 0.11470786693528087,
 (0, 3): 0.0668153104781061,
 (0, 4): 0.0,
 (0, 5): 0.0,
 (0, 6): 0.0,
 (0, 7): 0.0,
 (0, 8): 0.0,
 (0, 9): 0.0,
 (0, 10): 0.07905694150420949,
 (0, 11): 0.11785113019775793,
 (0, 12): 0.06454972243679027,
 (1, 2): 0.12725695259515554,
 (1, 3): 0.14824986333222026,
 (1, 4): 0.0,
 (1, 5): 0.0,
 (1, 6): 0.0,
 (1, 7): 0.0,
 (1, 8): 0.0,
 (1, 9): 0.0,
 (1, 10): 0.08770580193070293,
 (1, 11): 0.0,
 (1, 12): 0.0716114874039433,
 (2, 3): 0.12262786789699315,
 (2, 4): 0.0,
 (2, 5): 0.0,
 (2, 6): 0.0,
 (2, 7): 0.0,
 (2, 8): 0.061313933948496574,
 (2, 9): 0.1025978352085154,
 (2, 10): 0.0,
 (2, 11): 0.0,
 (2, 12): 0.0,
 (3, 4): 0.0,
 (3, 5): 0.0,
 (3, 6): 0.0,
 (3, 7): 0.0,
 (3, 8): 0.0,
 (3, 9): 0.0,
 (3, 10): 0.0,
 (3, 11): 0.0,
 (3, 12): 0.0,
 (4, 5): 0.0778498944161523,
 (4, 6): 0.09090909090909091,
 (4, 7): 0.08362420100070908,
 (4, 8): 0.08058229640253803,
 (4, 9): 0.13483997249264842,
 (4, 10): 0.09534625892455924,
 (4, 11): 0.0,

In [138]:
movies_similarity = dict(sorted(dic.items(), key=lambda item: item[1], reverse=True))
movies_similarity

{(0, 1): 0.20801257358446093,
 (5, 8): 0.20701966780270628,
 (7, 9): 0.1860521018838127,
 (10, 12): 0.1632993161855452,
 (10, 11): 0.14907119849998599,
 (1, 3): 0.14824986333222026,
 (7, 8): 0.14824986333222026,
 (5, 7): 0.1432229748078866,
 (4, 9): 0.13483997249264842,
 (6, 9): 0.13483997249264842,
 (1, 2): 0.12725695259515554,
 (2, 3): 0.12262786789699315,
 (8, 9): 0.11952286093343936,
 (0, 11): 0.11785113019775793,
 (5, 9): 0.11547005383792514,
 (0, 2): 0.11470786693528087,
 (2, 9): 0.1025978352085154,
 (4, 10): 0.09534625892455924,
 (4, 6): 0.09090909090909091,
 (1, 10): 0.08770580193070293,
 (4, 7): 0.08362420100070908,
 (6, 7): 0.08362420100070908,
 (4, 8): 0.08058229640253803,
 (6, 8): 0.08058229640253803,
 (0, 10): 0.07905694150420949,
 (4, 5): 0.0778498944161523,
 (5, 6): 0.0778498944161523,
 (1, 12): 0.0716114874039433,
 (9, 10): 0.07071067811865474,
 (0, 3): 0.0668153104781061,
 (0, 12): 0.06454972243679027,
 (2, 8): 0.061313933948496574,
 (11, 12): 0.06085806194501846,
 (9,

## Conclusion

The top 3 similar movies are:


1.  The lord of the rings and Inception
2.  It happened one night and gone with the wind
3.  Before Sunrise and Eternal Sunshine of the Spotless Mind




In [139]:
df_movie

,movie,review
0,The Lord of the Rings The Two Towers,remarkable display of fantasy action powerful ...
1,Inception,implanting stealing idea destroy gripping acti...
2,Spiderman Across the spider verse,mind bending wild action sequences intimate em...
3,The Dark Knight,Best live action portrayal beat organized crim...
4,Three colors red,mesmerising friendship turned love profound un...
5,It happened one night,Romantic comedy screwball comedy enduring tale...
6,In the Mood for Love,Neighbors solace bonding affair predicament sp...
7,Before Sunrise,blossoming love know each other chance encount...
8,Gone with the wind,epic romance greatest romantic film ever made ...
9,Eternal Sunshine of the Spotless Mind,Length people go finding love of life emotiona...
